
# Logistic Regression ( Titanic Dataset )

We will be working with the [Titanic Data Set from Kaggle](https://www.kaggle.com/c/titanic). This is a very famous data set and very often is a student's first step in machine learning! 

We'll be trying to predict a classification- survival or deceased.
Let's begin our understanding of implementing Logistic Regression in Python for classification.

We'll use a "semi-cleaned" version of the titanic data set, if you use the data set hosted directly on Kaggle, you may need to do some additional cleaning not shown in this lecture notebook.

## Import Libraries
Let's import some libraries to get started!

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling as pdp
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## The Data

Let's start by reading in the titanic_train.csv file into a pandas dataframe.

In [ ]:
train = pd.read_csv('../input/test-file/tested.csv')

In [ ]:
train.head(3)

# Exploratory Data Analysis

Let's begin some exploratory data analysis!

## Inspect Data with Pandas-Profiling

We can use ProfileReport to inspect all vars of data.

In [ ]:
pdp.ProfileReport(train)

Roughly 20 percent of the Age data is missing. The proportion of Age missing is likely small enough for reasonable replacement with some form of imputation. Looking at the Cabin column, it looks like we are just missing too much of that data to do something useful with at a basic level. We'll probably drop this later, or change it to another feature like "Cabin Known: 1 or 0"

Let's continue on by visualizing some more of the data! Check out the video for full explanations over these plots, this code is just to serve as reference.

___
## Data Cleaning
We want to fill in missing age data instead of just dropping the missing age data rows. One way to do this is by filling in the mean age of all the passengers (imputation).
However we can be smarter about this and check the average age by passenger class. For example:


In [ ]:
train_add = train.copy()
train_add['Pclass_Sex'] =  train_add['Sex'] + [str(s) for s in train_add.Pclass]

plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass_Sex',y='Age',data=train_add,palette='winter')

In [ ]:
train_add.groupby('Pclass_Sex').agg({'Age':np.mean})

We can see the wealthier passengers in the higher classes tend to be older, which makes sense. We'll use these average age values to impute based on Pclass for Age.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    Sex = cols[2]
    
    if pd.isnull(Age):

        if Pclass == 1 and Sex == 'male':
            return 40.5

        elif Pclass == 1 and Sex == 'female':
            return 41.3

        elif Pclass == 2 and Sex == 'male':
            return 30.9
        
        elif Pclass == 2 and Sex == 'female':
            return 24.4
        
        elif Pclass == 3 and Sex == 'male':
            return 24.5

        else:
            return 23.1

    else:
        return Age

Now apply that function!

In [ ]:
train['Age'] = train[['Age','Pclass', 'Sex']].apply(impute_age,axis=1)

Now let's check that heat map again!

In [ ]:
plt.figure(figsize=(15, 7))
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='coolwarm')

Great! Let's go ahead and drop the Cabin column and the row in Embarked that is NaN.

In [ ]:
train.drop('Cabin',axis=1,inplace=True);

In [ ]:
train.dropna(inplace=True);
train.head(3)

In [ ]:
del train_add

## Converting Categorical Features 

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [ ]:
train.info()

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train = pd.concat([train,sex,embark],axis=1)
train.drop(['Sex','Embarked','Name','Ticket','PassengerId'],axis=1,inplace=True)

In [ ]:
train.head(3)

Great! Our data is ready for our model!

# Building a Logistic Regression model

Let's start by splitting our data into a training set and test set (there is another test.csv file that you can play around with in case you want to use all this data for training).

## Train Test Split

In [ ]:
X = train[['Age', 'male', 'Pclass', 'Fare', 'Q', 'S']];
Y = train[['Survived']]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

## Train our model 

In [ ]:
lr = LogisticRegression() 
lr.fit(X_train, Y_train) 

In [ ]:
Y_pred = lr.predict(X_test)
print('precision = ', precision_score(y_true = Y_test, y_pred = Y_pred))

##### thanks for your support :)